# Chapter 13: Introduction to Modeling Libraries in Python

## 13.1 Interfacing Between pandas and Model Code

**Feature engineering** - Any data transformation or analytics that extract information from a raw dataset that may be useful in a modeling context.

Data aggregation and GroupBy tools are used often.

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [3]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [4]:
# Turn a DataFrame into a NumPy array
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [5]:
# Convert back to a DataFrame, pass a 2-D ndarray with optional column names
df2 = pd.DataFrame(data.values, columns=['one', 'two', 'three'])
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


> Note: `.values` is intended to be used when your data is homogeneous (ie. all the same type). If you have heterogeneous data, the result will be an ndarray of Python objects.

In [6]:
df3 = data.copy()
df3['strings'] = ['a', 'b', 'c', 'd', 'e']
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [7]:
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [8]:
# Use a subset of the columns
model_cols = ['x0', 'x1']
data.loc[:, model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

Suppose we had a non-numeric column in our example dataset:

> Note: Refer back to Chapter 12 for Categorical type and pd.get_dummies function.

In [9]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'],
                                  categories=['a', 'b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [10]:
"""To replace the 'category' column with dummy variables:
1. Create dummy variables
2. Drop the 'category' column
3. Join the result."""
dummies = pd.get_dummies(data.category, prefix='category')
data_with_dummies = data.drop('category', axis=1).join(dummies)
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


## 13.2 Creating Model Descriptions with Patsy

Patsy is a Python library for describing statistical models (especially linear models) with a small string-based "formula syntax."

Patsy's *formulas* look like: `y ~ x0 + x1`

The `patsy.dmatrices` function takes a formula string along with a dataset and produces design matrices for a linear model.

In [11]:
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [12]:
import patsy

In [13]:
y, X = patsy.dmatrices('y ~ x0 + x1', data)

In [14]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [15]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [16]:
# Patsy DesignMatrix instances are NumPy ndarrays with additional metadata
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [17]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

The `Intercept` term is a convention for linear models like ordinary least squares (OLS) regression.

In [18]:
# Suppress the intercept term
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [19]:
"""Patsy objects can be passed into numpy.linalg.lstsq to
perform ordinary least squares regression."""
coef, resid, _, _ = np.linalg.lstsq(X, y)

<ipython-input-19-e1fcb7b01fc5>:3: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid, _, _ = np.linalg.lstsq(X, y)


In [20]:
# The least squares coefficients
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [21]:
# Model metadata retained in design_info attribute
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

### 13.2.1 Data Transformations in Patsy Formulas

You can mix Python code into Patsy formulas.

In [22]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)

In [23]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [24]:
# Patsy has built-in functions for standardizing and centering
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)

In [25]:
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

When applying transformations like center and standardize, you should be careful when using the model to form predictions based on new data.

**Stateful transformations** - Using statistics like the mean or standard deviation of the **original** dataset to transform a new dataset.

`patsy.build_design_matrices()` can apply transformations to new out-of-sample data using the saved information from the original in-sample dataset.

In [26]:
new_data = pd.DataFrame({
    'x0': [6, 7, 8, 9],
    'x1': [3.1, -0.5, 0., 2.3],
    'y': [1, 2, 3, 4]})

In [27]:
new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [28]:
"""Plus (+) symbol does not mean addition in Patsy formulas.
Use special I wrapper function."""
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data) # add columns by name

In [29]:
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

### 13.2.2 Categorical Data and Patsy

In [30]:
data = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
    'key2': [0, 1, 0, 1, 0, 1, 0, 0],
    'v1': [1, 2, 3, 4, 5, 6, 7, 8],
    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})

y, X = patsy.dmatrices('v2 ~ key1', data)

In [31]:
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [32]:
# Omitting the intercept, columns will be included
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)

X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [33]:
# Numeric columns can be interpreted as categorical with C
y, X = patsy.dmatrices('v2 ~ C(key2)', data)

X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

In [34]:
data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [35]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)

X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [36]:
# Include interaction terms key1:key2 to be used in analysis of variance (ANOVA) models
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)

X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

## 13.3 Introduction to statsmodels

statsmodels models include:

- Linear models, generalized linear models, and robust linear models
- Linear mixed effects models
- Analysis of variance (ANOVA) methods
- Time series processes and state space models
- Generalized method of moments

### 13.3.1 Estimating Linear Models

In [37]:
# Linear models have 2 different main interfaces

import statsmodels.api as sm # Array-based
import statsmodels.formula.api as smf # Formula-based

In [38]:
# Generate a linear model from some random data

def dnorm(mean, variance, size=1):
    """ Helper function for generating normally distributed data with a particular mean and variance. """
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * np.random.randn(*size)

# For reproducibility
np.random.seed(12345)

N = 100
X = np.c_[dnorm(0, 0.4, size=N),
          dnorm(0, 0.6, size=N),
          dnorm(0, 0.2, size=N)]
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps

In [39]:
X[:5]

array([[-0.12946849, -1.21275292,  0.50422488],
       [ 0.30291036, -0.43574176, -0.25417986],
       [-0.32852189, -0.02530153,  0.13835097],
       [-0.35147471, -0.71960511, -0.25821463],
       [ 1.2432688 , -0.37379916, -0.52262905]])

In [40]:
y[:5]

array([ 0.42786349, -0.67348041, -0.09087764, -0.48949442, -0.12894109])

In [41]:
X_model = sm.add_constant(X) # Add an intercept column

X_model[:5]

array([[ 1.        , -0.12946849, -1.21275292,  0.50422488],
       [ 1.        ,  0.30291036, -0.43574176, -0.25417986],
       [ 1.        , -0.32852189, -0.02530153,  0.13835097],
       [ 1.        , -0.35147471, -0.71960511, -0.25821463],
       [ 1.        ,  1.2432688 , -0.37379916, -0.52262905]])

In [42]:
model = sm.OLS(y, X)  # Fit an ordinary least squares linear regression

results = model.fit()  # Contains estimated model parameters

results.params

array([0.17826108, 0.22303962, 0.50095093])

In [43]:
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.430
Model:                            OLS   Adj. R-squared (uncentered):              0.413
Method:                 Least Squares   F-statistic:                              24.42
Date:                Fri, 16 Jul 2021   Prob (F-statistic):                    7.44e-12
Time:                        20:48:08   Log-Likelihood:                         -34.305
No. Observations:                 100   AIC:                                      74.61
Df Residuals:                      97   BIC:                                      82.42
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [44]:
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])

data['y'] = y
data[:5]

,col0,col1,col2,y
0,-0.129468,-1.212753,0.504225,0.427863
1,0.302910,-0.435742,-0.254180,-0.673480
2,-0.328522,-0.025302,0.138351,-0.090878
3,-0.351475,-0.719605,-0.258215,-0.489494
4,1.243269,-0.373799,-0.522629,-0.128941


In [45]:
# Using statsmodels formula API and Patsy formula strings

results = smf.ols('y ~ col1 + col2', data=data).fit()

In [46]:
results.params

Intercept    0.038450
col1         0.219154
col2         0.532058
dtype: float64

In [47]:
results.tvalues

Intercept    1.039562
col1         4.504607
col2         6.215285
dtype: float64

In [48]:
results.predict(data[:5])

0    0.040948
1   -0.192283
2    0.106516
3   -0.256639
4   -0.321538
dtype: float64

### 13.3.2 Estimating Time Series Processes

statsmodels also has models for time series analysis sch as:

- Autoregressive processes
- Kalman filtering and other state space models
- Multivariate autoregressive models

In [50]:
# Simulate time series data with an autoregressive structure and noise

init_x = 4

import random
values = [init_x, init_x]
N = 1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

This data has an "Autoregressive" AR(2) structure (two lags) with parameters 0.8 and -0.4. You may not know the number of lagged terms to include, so you can fit the model with some larger number of lags.

In [52]:
MAXLAGS = 5

model = sm.tsa.AR(values)
results = model.fit(MAXLAGS)

results.params

c:\Users\Productivity\Coding\Python_Data_Analysis\Notes\venv\lib\site-packages\statsmodels\tsa\ar_model.py:791: FutureWarning: 
statsmodels.tsa.AR has been deprecated in favor of statsmodels.tsa.AutoReg and
statsmodels.tsa.SARIMAX.

AutoReg adds the ability to specify exogenous variables, include time trends,
and add seasonal dummies. The AutoReg API differs from AR since the model is
treated as immutable, and so the entire specification including the lag
length must be specified when creating the model. This change is too
substantial to incorporate into the existing AR api. The function
ar_select_order performs lag length selection for AutoReg models.

AutoReg only estimates parameters using conditional MLE (OLS). Use SARIMAX to
estimate ARX and related models using full MLE via the Kalman Filter.

To silence this warning and continue using AR until it is removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.ar_model.AR', FutureWarning)

  warnings.warn(AR_

array([ 0.00791554,  0.71074082, -0.31689896, -0.06724519,  0.0041215 ,
       -0.00079061])

## 13.4 Introduction to scikit-learn

As an example, let's use the Kaggle Titanic dataset.

In [53]:
train = pd.read_csv('datasets/titanic/train.csv')
test = pd.read_csv('datasets/titanic/test.csv')

In [54]:
train[:4]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [58]:
# statsmodels & scikit-learn cannot be fed missing data

train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [59]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [60]:
# Use median of training set to fill the nulls in both tables

impute_value = train['Age'].median()

train['Age'] = train['Age'].fillna(impute_value)
test['Age'] = test['Age'].fillna(impute_value)

In [61]:
# Add IsFemale as encoded version of the 'Sex' column

train['IsFemale'] = (train['Sex'] == 'female').astype(int)
test['IsFemale'] = (train['Sex'] == 'female').astype(int)

In [62]:
# Decide model variables and create NumPy arrays

predictors = ['Pclass', 'IsFemale', 'Age']

X_train = train[predictors].values
X_test = test[predictors].values
y_train = train['Survived'].values

In [63]:
X_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

In [64]:
y_train[:5]

array([0, 1, 1, 1, 0], dtype=int64)

In [65]:
# Use LogisticRegression model and create model instance

from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

In [66]:
# Fit model to training data

model.fit(X_train, y_train)

LogisticRegression()

In [67]:
# Form predictions for test dataset

y_predict = model.predict(X_test)

y_predict[:10]

array([0, 0, 1, 1, 0, 0, 0, 0, 1, 1], dtype=int64)

In [75]:
# Compute accuracy percentage

# If given the true values for the test dataset
# (y_true == y_predict).mean()

There are many additional layers of complexity such as:

- Parameters to tune
- Cross-validation for parameter tuning to avoid overfitting

Cross-validation works by splitting the training data to simulate out-of-sample prediction.

In [76]:
from sklearn.linear_model import LogisticRegressionCV

model_cv = LogisticRegressionCV(10)

c:\Users\Productivity\Coding\Python_Data_Analysis\Notes\venv\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass Cs=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [77]:
model_cv.fit(X_train, y_train)

LogisticRegressionCV()

In [78]:
# Cross-validation

from sklearn.model_selection import cross_val_score

model = LogisticRegression(C=10)

scores = cross_val_score(model, X_train, y_train, cv=4)

In [80]:
scores

array([0.77578475, 0.79820628, 0.77578475, 0.78828829])